# Minimal Q&A Agent Using IBM Granite via Ollama and Semantic Kernel

This recipe demonstrates how to create a minimal Q&A agent using IBM Granite models through Ollama and Microsoft's Semantic Kernel framework.

## Overview

In this notebook, you'll learn how to:
- Set up Ollama with IBM Granite models
- Configure Semantic Kernel to work with Ollama
- Create a simple Q&A agent
- Implement chat history management
- Build interactive conversations

## Prerequisites

1. **Ollama**: Download and install Ollama from [ollama.ai](https://ollama.ai)
2. **IBM Granite Model**: Pull an IBM Granite model using Ollama
3. **Python Dependencies**: Install required Python packages

### Install Ollama and Pull Granite Model

First, ensure Ollama is running and pull an IBM Granite model:

```bash
# Install Ollama (macOS)
# Download from https://ollama.ai

# Pull IBM Granite 3.0 2B model
ollama pull granite3-dense:2b

# Alternative models:
# ollama pull granite3-dense:8b
# ollama pull granite3-moe:3b
```

## Install Required Dependencies

In [ ]:
# Install Semantic Kernel and other dependencies
!pip install semantic-kernel aiohttp python-dotenv

## Import Required Libraries

In [ ]:
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.core_plugins.conversation_summary_plugin import ConversationSummaryPlugin
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Configure Semantic Kernel with Ollama

Set up the Semantic Kernel to use IBM Granite via Ollama:

In [ ]:
# Configuration
OLLAMA_HOST = "http://localhost:11434"  # Default Ollama host
MODEL_NAME = "granite3-dense:2b"  # IBM Granite model name

# Initialize Semantic Kernel
kernel = Kernel()

# Add Ollama chat completion service
chat_completion = OllamaChatCompletion(
    ai_model_id=MODEL_NAME,
    host=OLLAMA_HOST,
    service_id="ollama-granite"
)

kernel.add_service(chat_completion)

print(f"✅ Semantic Kernel configured with Ollama")
print(f"📍 Host: {OLLAMA_HOST}")
print(f"🤖 Model: {MODEL_NAME}")

## Create Chat History Manager

Set up chat history to maintain conversation context:

In [ ]:
# Initialize chat history
chat_history = ChatHistory()

# Add system message to set the agent's behavior
chat_history.add_system_message(
    "You are a helpful AI assistant powered by IBM Granite. "
    "You provide accurate, concise, and helpful responses to user questions. "
    "If you're unsure about something, you'll say so rather than guessing."
)

print("✅ Chat history initialized with system prompt")

## Simple Q&A Function

Create a function to handle questions and maintain conversation context:

In [ ]:
async def ask_question(question: str, chat_history: ChatHistory) -> str:
    """
    Ask a question to the Granite model and get a response.
    
    Args:
        question (str): The user's question
        chat_history (ChatHistory): The conversation history
    
    Returns:
        str: The assistant's response
    """
    try:
        # Add user message to history
        chat_history.add_user_message(question)
        
        # Get chat completion service
        chat_service = kernel.get_service("ollama-granite")
        
        # Generate response
        response = await chat_service.get_chat_message_content(
            chat_history=chat_history,
            settings=None
        )
        
        # Add assistant response to history
        chat_history.add_assistant_message(str(response))
        
        return str(response)
        
    except Exception as e:
        error_msg = f"Error getting response: {str(e)}"
        logger.error(error_msg)
        return error_msg

print("✅ Q&A function defined")

## Test the Q&A Agent

Let's test our minimal Q&A agent with some sample questions:

In [ ]:
# Test question 1
question1 = "What is IBM Granite?"
print(f"❓ User: {question1}")

response1 = await ask_question(question1, chat_history)
print(f"🤖 Assistant: {response1}")
print("-" * 50)

In [ ]:
# Test question 2 - follow-up question to test context retention
question2 = "What are its main capabilities?"
print(f"❓ User: {question2}")

response2 = await ask_question(question2, chat_history)
print(f"🤖 Assistant: {response2}")
print("-" * 50)

In [ ]:
# Test question 3 - technical question
question3 = "How can I use Semantic Kernel with Ollama?"
print(f"❓ User: {question3}")

response3 = await ask_question(question3, chat_history)
print(f"🤖 Assistant: {response3}")
print("-" * 50)

## Interactive Chat Loop

Create an interactive chat interface:

In [ ]:
async def interactive_chat():
    """
    Run an interactive chat session with the Q&A agent.
    Type 'quit', 'exit', or 'bye' to end the conversation.
    """
    print("🚀 Starting interactive chat with IBM Granite via Ollama!")
    print("💡 Type 'quit', 'exit', or 'bye' to end the conversation.\n")
    
    while True:
        try:
            # Get user input
            user_input = input("❓ You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', '']:
                print("👋 Goodbye! Thanks for chatting!")
                break
            
            # Get and display response
            print("🤔 Thinking...")
            response = await ask_question(user_input, chat_history)
            print(f"🤖 Assistant: {response}\n")
            
        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ An error occurred: {e}")
            continue

print("✅ Interactive chat function defined")
print("🔧 Run the next cell to start an interactive chat session!")

In [ ]:
# Start interactive chat (uncomment to run)
# await interactive_chat()

## View Chat History

Examine the conversation history:

In [ ]:
def display_chat_history(chat_history: ChatHistory):
    """
    Display the conversation history in a formatted way.
    """
    print("📜 Chat History:")
    print("=" * 60)
    
    for i, message in enumerate(chat_history.messages):
        role = message.role.value.upper()
        content = str(message.content)[:100] + "..." if len(str(message.content)) > 100 else str(message.content)
        
        if role == "SYSTEM":
            emoji = "⚙️"
        elif role == "USER":
            emoji = "❓"
        else:
            emoji = "🤖"
            
        print(f"{emoji} {role}: {content}")
        print("-" * 40)

# Display current chat history
display_chat_history(chat_history)

## Advanced Features

### Add Conversation Summary Plugin

For longer conversations, you can use Semantic Kernel's conversation summary plugin:

In [ ]:
# Add conversation summary plugin
kernel.add_plugin(ConversationSummaryPlugin(), plugin_name="conversation")

async def summarize_conversation(chat_history: ChatHistory) -> str:
    """
    Summarize the current conversation.
    """
    try:
        # Get the conversation summary function
        summary_function = kernel.get_function("conversation", "SummarizeConversation")
        
        # Convert chat history to string
        conversation_text = "\n".join([f"{msg.role.value}: {msg.content}" for msg in chat_history.messages])
        
        # Generate summary
        result = await kernel.invoke(summary_function, input=conversation_text)
        
        return str(result)
        
    except Exception as e:
        return f"Error generating summary: {str(e)}"

print("✅ Conversation summary plugin added")

In [ ]:
# Generate conversation summary
if len(chat_history.messages) > 1:
    print("📝 Generating conversation summary...")
    summary = await summarize_conversation(chat_history)
    print(f"📋 Summary: {summary}")
else:
    print("💡 Not enough conversation to summarize yet. Ask a few questions first!")

## Configuration Options

### Model Parameters

You can customize the model's behavior with various parameters:

In [ ]:
from semantic_kernel.connectors.ai.ollama import OllamaChatPromptExecutionSettings

# Create custom execution settings
execution_settings = OllamaChatPromptExecutionSettings(
    temperature=0.7,  # Controls randomness (0.0 = deterministic, 1.0 = very random)
    top_p=0.9,       # Controls diversity of response
    max_tokens=500,  # Maximum tokens in response
)

async def ask_question_with_settings(question: str, chat_history: ChatHistory, settings=None) -> str:
    """
    Ask a question with custom execution settings.
    """
    try:
        chat_history.add_user_message(question)
        
        chat_service = kernel.get_service("ollama-granite")
        
        response = await chat_service.get_chat_message_content(
            chat_history=chat_history,
            settings=settings or execution_settings
        )
        
        chat_history.add_assistant_message(str(response))
        return str(response)
        
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Custom execution settings configured")
print(f"🌡️ Temperature: {execution_settings.temperature}")
print(f"🎯 Top-p: {execution_settings.top_p}")
print(f"📏 Max tokens: {execution_settings.max_tokens}")

## Test Different Models

If you have multiple Granite models available, you can easily switch between them:

In [ ]:
# Available IBM Granite models in Ollama
available_models = [
    "granite3-dense:2b",
    "granite3-dense:8b", 
    "granite3-moe:3b",
]

def create_chat_service(model_name: str):
    """
    Create a new chat service with a different model.
    """
    return OllamaChatCompletion(
        ai_model_id=model_name,
        host=OLLAMA_HOST,
        service_id=f"ollama-{model_name.replace(':', '-')}"
    )

print("🔄 Available IBM Granite models:")
for model in available_models:
    print(f"  • {model}")
print("\n💡 You can switch models by creating a new chat service and kernel.")

## Troubleshooting

### Common Issues and Solutions

1. **Ollama not running**: Make sure Ollama is installed and running
2. **Model not found**: Ensure you've pulled the IBM Granite model with `ollama pull granite3-dense:2b`
3. **Connection issues**: Verify the Ollama host URL (default: http://localhost:11434)
4. **Memory issues**: Try using a smaller model like `granite3-dense:2b` instead of `granite3-dense:8b`

In [ ]:
# Test Ollama connection
import aiohttp
import json

async def test_ollama_connection():
    """
    Test if Ollama is running and the model is available.
    """
    try:
        async with aiohttp.ClientSession() as session:
            # Test if Ollama is running
            async with session.get(f"{OLLAMA_HOST}/api/tags") as response:
                if response.status == 200:
                    data = await response.json()
                    models = [model['name'] for model in data.get('models', [])]
                    
                    print("✅ Ollama is running")
                    print(f"📦 Available models: {models}")
                    
                    if MODEL_NAME in models:
                        print(f"✅ Model '{MODEL_NAME}' is available")
                    else:
                        print(f"❌ Model '{MODEL_NAME}' not found. Please run: ollama pull {MODEL_NAME}")
                else:
                    print(f"❌ Ollama responded with status {response.status}")
                    
    except Exception as e:
        print(f"❌ Cannot connect to Ollama: {e}")
        print("💡 Make sure Ollama is installed and running")

# Run connection test
await test_ollama_connection()

## Next Steps

Now that you have a working Q&A agent, you can:

1. **Add more plugins**: Integrate other Semantic Kernel plugins for enhanced functionality
2. **Create custom functions**: Build domain-specific functions for your use case
3. **Add memory**: Implement persistent memory to remember conversations across sessions
4. **Build a web interface**: Create a web app using frameworks like Streamlit or FastAPI
5. **Integrate with RAG**: Combine with retrieval-augmented generation for knowledge-based responses

### Additional Resources

- [Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Ollama Documentation](https://ollama.ai/docs)
- [IBM Granite Models](https://ollama.com/blog/ibm-granite)
- [Granite Snack Cookbook](https://github.com/ibm-granite-community/granite-snack-cookbook)

---

🎉 **Congratulations!** You've successfully created a minimal Q&A agent using IBM Granite via Ollama and Semantic Kernel!